In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
df = pd.read_excel('data/online-retail.xlsx')
df.dropna(axis=0,subset=['InvoiceNo'],inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [5]:
# remove credits by dropping rows where InvoiceNo == 'C'

# df[df.InvoiceNo.str.contains('C',na=False)].head()
df = df[~df['InvoiceNo'].str.contains('C')]

In [6]:
# Time for the market basket analysis
# First, groupby columns we want to consider

market_basket = df[df['Country'] == "United Kingdom"].groupby(['InvoiceNo','Description'])['Quantity']

In [7]:
# hot encode the data and get one tx per row.
market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [8]:
market_basket.head()

Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def encode_data(datapoint):
    if datapoint <= 0:
        return 0
    if datapoint >= 1:
        return 1

In [10]:
market_basket = market_basket.applymap(encode_data)

In [16]:
# to find out which items are frequently purchased together, we apply the mlxtend `apriori` function to our dataset
market_basket.convert_dtypes(bool)
itemsets = apriori(market_basket, min_support=0.03, use_colnames=True)

/home/thowe/miniconda3/envs/cudnn-1/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [17]:
# build rules using the mlxtend `association_rules` function

rules = association_rules(itemsets,metric="lift",min_threshold=0.5)

In [18]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.049818,0.046925,0.030159,0.605376,12.900874,0.027821,2.415149
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.046925,0.049818,0.030159,0.642694,12.900874,0.027821,2.659296
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037658,0.050032,0.030909,0.820768,16.404818,0.029024,5.300218
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.050032,0.037658,0.030909,0.617773,16.404818,0.029024,2.517724
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.051264,0.050032,0.037551,0.732497,14.640537,0.034986,3.551247
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.050032,0.051264,0.037551,0.750535,14.640537,0.034986,3.803087
6,(JUMBO BAG RED RETROSPOT),(JUMBO BAG BAROQUE BLACK WHITE),0.103814,0.048747,0.030534,0.294118,6.033613,0.025473,1.347609
7,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.048747,0.103814,0.030534,0.626374,6.033613,0.025473,2.398615
8,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.103814,0.062085,0.042051,0.405057,6.524245,0.035605,1.576478
9,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.062085,0.103814,0.042051,0.677308,6.524245,0.035605,2.777218


## When can MLxtend not be used?

sometimes pandas has issues unfolding large datasets.

In [21]:
from itertools import combinations, groupby
from collections import Counter

df_manual = df[df['Country'] =="United Kingdom"]


orders = df_manual.set_index('InvoiceNo')['StockCode']

statistics = orders.value_counts().to_frame("frequency")
statistics['support'] = statistics / len(set(orders.index)) * 100

min_support = 0.03 # same value used above

items_above_support = statistics[statistics['support'] >= min_support].index
orders_above_support = orders[orders.isin(items_above_support)]

order_counts = orders.index.value_counts()
orders_over_two_index = order_counts[order_counts>=2].index
orders_over_two = orders[orders.index.isin(orders_over_two_index)]

In [22]:
statistics = orders_over_two.value_counts().to_frame("frequency")
statistics['support'] = statistics / len(set(orders_over_two.index)) * 100


In [24]:
def itemset_generator(orders):
    orders = orders.reset_index().values
    for order_id, order_object in groupby(orders, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
        for item_pair in combinations(item_list, 2):
            yield item_pair


itemsets_gen = itemset_generator(orders_over_two)
itemsets = pd.Series(Counter(itemsets_gen)).to_frame("frequencyAC")
itemsets['supportAC'] = itemsets['frequencyAC']/len(orders_over_two_index) * 100
itemsets = itemsets[itemsets['supportAC'] >= min_support]

In [25]:
# create table of association rules
itemsets = itemsets.reset_index().rename(columns={'level_0': 'antecedents', 'level_1': 'consequents'})
itemsets = (itemsets
     .merge(statistics.rename(columns={'freq': 'freqA', 'support': 'antecedent support'}), left_on='antecedents', right_index=True)
     .merge(statistics.rename(columns={'freq': 'freqC', 'support': 'consequents support'}), left_on='consequents', right_index=True))

itemsets['confidenceAtoC'] = itemsets['supportAC'] / itemsets['antecedent support']
itemsets['confidenceCtoA'] = itemsets['supportAC'] / itemsets['consequents support']
itemsets['lift'] = itemsets['supportAC'] / (itemsets['antecedent support'] * itemsets['consequents support'])
itemsets=itemsets[['antecedents', 'consequents','antecedent support', 'consequents support', 'confidenceAtoC','lift']]

In [26]:
rules = itemsets
rules_over_50 = rules[(rules.confidenceAtoC >0.50)]
rules_over_50.set_index('antecedents',inplace=True)
rules_over_50.reset_index(inplace=True)
rules_over_50=rules_over_50.sort_values('lift', ascending=False)

In [27]:
rules_over_50

,antecedents,consequents,antecedent support,consequents support,confidenceAtoC,lift
1841,48173c,85099f,0.042296,0.060423,0.714286,11.821429
1881,90082D,90082B,0.036254,0.072508,0.833333,11.493056
1888,20698,20697,0.054381,0.078550,0.888889,11.316239
1281,90129D,90129E,0.042296,0.084592,0.857143,10.132653
1880,37444C,72051S,0.054381,0.084592,0.666667,7.880952
...,...,...,...,...,...,...
1152,35915C,85123A,0.356495,13.021148,0.525424,0.040352
1135,85039A,85123A,0.966767,13.021148,0.518750,0.039839
1145,85093,85123A,0.513595,13.021148,0.517647,0.039754
1151,84638,85123A,0.199396,13.021148,0.515152,0.039563
